<a href="https://colab.research.google.com/github/sina-salmanpour/Loan_Defaults_prediction/blob/main/Loan_Defaults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

import warnings
from IPython.core.interactiveshell import InteractiveShell

Libraries initial settings

In [32]:
## Following lines are used to control how warnings are displayed.

warnings.filterwarnings('ignore')
warnings.simplefilter(action="ignore", category=FutureWarning)  # warnings about features that will change in the future.
warnings.simplefilter(action="ignore", category=Warning)        # more general category for warnings.


## Setting below is related to how IPython displays output.
## The results of all statements in a cell that would normally be displayed are shown

InteractiveShell.ast_node_interactivity = "all"

# matplotlib plots are displayed directly within the notebook output

%matplotlib inline
np.set_printoptions(suppress=True)
plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams['figure.dpi'] = 100

sns.set_style("whitegrid")

def set_seed(seed=42):
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

pd.set_option("display.width", 100)
pd.set_option("display.max_columns", 60)
pd.set_option("display.max_rows", 25)
pd.set_option("display.float_format", lambda x: "%.3f" % x)


Loading Dataset

In [11]:
df = pd.read_csv('/content/drive/MyDrive/Loan Defaults/Dataset ARCHIVE/Training Data.csv')

Initial inspection

In [12]:
print("Dataset Shape:", df.shape)
print("\nDataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())
print("\nTarget variable distribution:")
print(df['Risk_Flag'].value_counts())

Dataset Shape: (252000, 13)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252000 entries, 0 to 251999
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Id                 252000 non-null  int64 
 1   Income             252000 non-null  int64 
 2   Age                252000 non-null  int64 
 3   Experience         252000 non-null  int64 
 4   Married/Single     252000 non-null  object
 5   House_Ownership    252000 non-null  object
 6   Car_Ownership      252000 non-null  object
 7   Profession         252000 non-null  object
 8   CITY               252000 non-null  object
 9   STATE              252000 non-null  object
 10  CURRENT_JOB_YRS    252000 non-null  int64 
 11  CURRENT_HOUSE_YRS  252000 non-null  int64 
 12  Risk_Flag          252000 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 25.0+ MB
None

First 5 rows:
   Id   Income  Age  Experience Married/Single House_Ow

In [19]:
# Check for missing values
missing_values = df.isnull().sum()
if missing_values.sum() == 0:
    print("No missing values found in the dataset.")
else:
    print("Missing Values:")
    print(missing_values[missing_values > 0])

    # Calculate missing value percentages
    missing_percent = (missing_values / len(df)) * 100
    print("\nMissing Value Percentages:")
    print(missing_percent[missing_percent > 0])

No missing values found in the dataset.


In [20]:
# Check for duplicate rows
duplicate_rows = df.duplicated().sum()
if duplicate_rows == 0:
    print("\nNo duplicate rows found in the dataset.")
else:
    print(f"\nNumber of duplicate rows found: {duplicate_rows}")


No duplicate rows found in the dataset.
